In [ ]:
%run 00_common.ipynb

In [ ]:
post_login()
ws = load_workspace()

In [ ]:
# Local web service port
port=8890

### Deploy as a Local Web Service

In [ ]:
from azureml.core.model import Model
from azureml.core.image import ContainerImage

from azureml.core.webservice import LocalWebservice, Webservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(runtime= "python", 
                                   entry_script="iot_score.py",
                                   conda_file="myenv.yml",
                                   extra_docker_file_steps="/home/hungap/development/azureml/docker/set-proxy")
# image_config = ContainerImage.image_configuration(runtime= "python",
#                                  execution_script="iot_score.py",
#                                  conda_file="myenv.yml",
#                                  docker_file="localDockerFile",
#                                  tags = {'area': "iot", 'type': "classification"},
#                                  description = "IOT Edge anomaly detection demo")
deployment_config = LocalWebservice.deploy_configuration(port=port)
service = Model.deploy(ws, "anomaly_ws", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

### Test Web Service

In [ ]:
import requests
import json

headers = {'Content-Type': 'application/json'}

local_web_service = True
scoring_uri = "http://localhost:" + port+ "/score"

test_sample = json.dumps({ "machine": { "temperature": 31.16469009, "pressure": 2.158002669 }, \
                          "ambient": { "temperature": 21.17794693, "humidity": 25 },\
                          "timeCreated": "2017-10-27T18:14:02.4911177Z" })

response = requests.post(scoring_uri, data=test_sample, headers=headers)
print(response.status_code)
print(response.content)

### Remove Web Service

In [ ]:
service.delete()